In [ ]:
### A CUSTOM BUILT FUNCTION THAT COMPUTES INFORMATION VALUE OF ALL PREDICTORS

### This function takes as an argument the DATAFRAME, the TARGET VARIABLE NAME and the PREDICTOR VARIABLE NAME
### after inputting these values, the function would output the INFORMATION VALUE of the predictor with respect to the target
### this would give us a sense of how informative the variable would be prior to subjecting it to modeling. 

def iv(data, target, predictor):
    
    # for computing IV for interval variables, the interval variables first need to be BINNED
    # that is what is happening here. We are creating BINNED versions of LOG TRANSFORMS of an interval variable
    
    if predictor in list(set(list(data.columns)) - set(list(data.select_dtypes(include=['int', 'float']).columns))):
        d = pd.DataFrame([df[target], df[predictor]]).T
    elif predictor in list(df.select_dtypes(include=['int', 'float']).columns):
        bc = pd.qcut(np.log10(df[predictor]+3), q=6, labels=list(range(6)))
        d = pd.DataFrame([df[target], bc]).T
        
    # count the total number of GOODS (response = 1) and BADS (response = 0)
    bads = d[d[target]==0][target].count()
    goods = d[d[target]==1][target].count()
    
    g = pd.Series(d[d[target]==1][predictor].value_counts(), name='yes')
    g = g/goods
    
    b = pd.Series(d[d[target]==0][predictor].value_counts(), name='no')
    b = b/bads
    dd = pd.DataFrame([g, b]).T
    
    # apply the IV mathematical formula and return as output
    IV = (dd['yes'] - dd['no'])*np.log10(dd['yes']/dd['no'])
    return IV.sum()

In [ ]:
# how to call the IV function: enter as arguments the DATAFRAME, the TARGET COLUMN NAME in QUOTES, the PREDICTOR variable
# for which you want to compute the IV.

infoval = iv(df, 'Target', pred)

In [ ]:
############################
############################
############################

In [ ]:
### THIS FUNCTION IDENTIFIES ALL CATEGORICAL COLUMNS AND COMPUTES AND JOINS THEIR DUMMY ENCODED MATRICES
### ADDITIONALLY IT ALSO STANDARDIZES THE NUMERICAL VARIABLES AND COMPILES A FINAL DATASET

### This function takes as argument the input DATAFRAME and the TARGET COLUMN NAME
### it would then automatically identify the CATEGORICAL columns and NUMERICAL columns in your predictors
### after this, it would DUMMY ENCODE the categorical columns and STANDARD NORMALIZE all the interval predictors

### as an output, it gives out the FINAL combined dataframe of all the transformed predictors separately
### and it also outputs the target variable column as a separate SERIES. 

### both these separate entities can be readily passed on to any algorithm for modeling

def final_frame(xx, target):
    
    X = xx.copy()
    cat_cols = list(set(list(X.columns)) - set(list(X.select_dtypes(include=['int', 'float']).columns)))
    q_cols = list(X.select_dtypes(include=['int', 'float']).columns)
    cat_cols.remove(target)
    initialized = cat_cols[0]
    cat_cols.remove(initialized)
    frame = pd.get_dummies(X[initialized], drop_first=True)
    
    Y = X[target]
    X.pop(target)

    # this loop makes dummy columns from each categorical variable in the data
    for column in list(X.columns):
        if column in cat_cols:
            frame = frame.join(pd.get_dummies(X[column], drop_first=True))
        else:
            pass
        
    # this look standard normalizes each of the numerical columns in the data    
    Qcols = []
    for col in q_cols:
        X[col] = (X[col] - X[col].mean())/X[col].std()
    
    frame2 = X[q_cols]
    finalframe = pd.concat([frame2, frame], axis=1)
    
    # the return value is a DATAFRAME for all the predictors (dummy encoded and transformed) AND a SERIES for target variable
    return finalframe, Y

In [ ]:
# example function call - pass the DATAFRAME and TARGET VARIABLE NAME and get as outputs - 
# - DATAFRAME of transformed predictors and SERIES of target variable.

x_data, y = final_frame(data, 'Target')